In [ ]:
import os
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

# Use the provided script to load STL-10 data
def read_labels(path_to_labels):
    with open(path_to_labels, 'rb') as f:
        labels = np.fromfile(f, dtype=np.uint8)
        return labels

def read_all_images(path_to_data):
    with open(path_to_data, 'rb') as f:
        everything = np.fromfile(f, dtype=np.uint8)
        images = np.reshape(everything, (-1, 3, 96, 96))
        images = np.transpose(images, (0, 3, 2, 1))
        return images

# Define a custom PyTorch dataset for STL-10
class STL10Dataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx] - 1  # Convert to 0-based index
        if self.transform:
            image = self.transform(image)
        return image, label

# Load STL-10 data
DATA_DIR = '/content'
DATA_PATH = os.path.join(DATA_DIR, 'train_X.bin')
LABEL_PATH = os.path.join(DATA_DIR, 'train_y.bin')

images = read_all_images(DATA_PATH)
labels = read_labels(LABEL_PATH)

# Define transformations
stl10_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((96, 96)),
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Create dataset and data loader
stl10_dataset = STL10Dataset(images, labels, transform=stl10_transform)
stl10_loader = DataLoader(stl10_dataset, batch_size=64, shuffle=True)

In [ ]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define a simple CNN
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 24 * 24, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Initialize model, loss function, and optimizer
model = CNN(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Pre-train on STL-10
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(stl10_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(stl10_loader):.4f}")

100%|██████████| 79/79 [01:34<00:00,  1.20s/it]


Epoch [1/10], Loss: 2.0383


100%|██████████| 79/79 [01:34<00:00,  1.20s/it]


Epoch [2/10], Loss: 1.5275


100%|██████████| 79/79 [01:35<00:00,  1.21s/it]


...

Epoch [8/10], Loss: 0.1267


100%|██████████| 79/79 [01:42<00:00,  1.30s/it]


Epoch [9/10], Loss: 0.0610


100%|██████████| 79/79 [01:50<00:00,  1.40s/it]

Epoch [10/10], Loss: 0.0252


In [ ]:
import torchvision

# Preprocessing for MNIST
mnist_transform = transforms.Compose([
    transforms.Resize((96, 96)),
    transforms.Grayscale(num_output_channels=3),  # Convert to 3 channels
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

# Load MNIST dataset
mnist_train = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=mnist_transform)
mnist_test = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=mnist_transform)

# Create data loaders
mnist_train_loader = DataLoader(mnist_train, batch_size=64, shuffle=True)
mnist_test_loader = DataLoader(mnist_test, batch_size=64, shuffle=False)

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9.91M/9.91M [00:00<00:00, 20.6MB/s]


Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28.9k/28.9k [00:00<00:00, 610kB/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1.65M/1.65M [00:00<00:00, 4.83MB/s]


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4.54k/4.54k [00:00<00:00, 1.69MB/s]

Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



In [ ]:
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm

# Define a simple CNN
class CNN(nn.Module):
    def __init__(self, num_classes=10):
        super(CNN, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(3, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(64 * 24 * 24, 128),
            nn.ReLU(),
            nn.Linear(128, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

# Initialize model, loss function, and optimizer
model = CNN(num_classes=10)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Pre-train on STL-10
num_epochs = 10
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(stl10_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(stl10_loader):.4f}")

100%|██████████| 79/79 [01:37<00:00,  1.23s/it]


Epoch [1/10], Loss: 1.8855


100%|██████████| 79/79 [01:37<00:00,  1.24s/it]


Epoch [2/10], Loss: 1.4327


100%|██████████| 79/79 [01:37<00:00,  1.23s/it]


...

Epoch [8/10], Loss: 0.0807


100%|██████████| 79/79 [01:41<00:00,  1.28s/it]


Epoch [9/10], Loss: 0.0279


100%|██████████| 79/79 [01:51<00:00,  1.41s/it]

Epoch [10/10], Loss: 0.0121


In [ ]:
# Freeze all layers except the classifier
for param in model.features.parameters():
    param.requires_grad = False

# Replace the classifier for MNIST (10 classes)
model.classifier = nn.Sequential(
    nn.Linear(64 * 24 * 24, 128),
    nn.ReLU(),
    nn.Linear(128, 10),
)

# Train only the classifier
optimizer = optim.Adam(model.classifier.parameters(), lr=0.001)

for epoch in range(5):  # Fewer epochs for fine-tuning
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(mnist_train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/5], Loss: {running_loss/len(mnist_train_loader):.4f}")

100%|██████████| 938/938 [09:46<00:00,  1.60it/s]


Epoch [1/5], Loss: 0.1768


100%|██████████| 938/938 [10:01<00:00,  1.56it/s]


Epoch [2/5], Loss: 0.0778


100%|██████████| 938/938 [10:00<00:00,  1.56it/s]


Epoch [3/5], Loss: 0.0617


100%|██████████| 938/938 [09:59<00:00,  1.56it/s]


Epoch [4/5], Loss: 0.0483


100%|██████████| 938/938 [10:00<00:00,  1.56it/s]

Epoch [5/5], Loss: 0.0427


In [ ]:
# Freeze the first few layers
for i, param in enumerate(model.features.parameters()):
    if i < 4:  # Freeze the first two convolutional layers
        param.requires_grad = False

# Train the rest of the model
optimizer = optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(mnist_train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/5], Loss: {running_loss/len(mnist_train_loader):.4f}")

100%|██████████| 938/938 [10:29<00:00,  1.49it/s]


Epoch [1/5], Loss: 0.0407


100%|██████████| 938/938 [10:31<00:00,  1.48it/s]


Epoch [2/5], Loss: 0.0332


100%|██████████| 938/938 [10:39<00:00,  1.47it/s]


Epoch [3/5], Loss: 0.0292


100%|██████████| 938/938 [10:34<00:00,  1.48it/s]


Epoch [4/5], Loss: 0.0274


100%|██████████| 938/938 [10:38<00:00,  1.47it/s]

Epoch [5/5], Loss: 0.0238


In [ ]:
# Unfreeze all layers
for param in model.parameters():
    param.requires_grad = True

# Fine-tune the entire model
optimizer = optim.Adam(model.parameters(), lr=0.0001)  # Lower learning rate

for epoch in range(5):
    model.train()
    running_loss = 0.0
    for images, labels in tqdm(mnist_train_loader):
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    print(f"Epoch [{epoch+1}/5], Loss: {running_loss/len(mnist_train_loader):.4f}")

100%|██████████| 938/938 [19:51<00:00,  1.27s/it]


Epoch [1/5], Loss: 0.0051


100%|██████████| 938/938 [19:35<00:00,  1.25s/it]


Epoch [2/5], Loss: 0.0030


100%|██████████| 938/938 [19:33<00:00,  1.25s/it]


Epoch [3/5], Loss: 0.0017


100%|██████████| 938/938 [19:35<00:00,  1.25s/it]


Epoch [4/5], Loss: 0.0014


100%|██████████| 938/938 [19:42<00:00,  1.26s/it]

Epoch [5/5], Loss: 0.0010


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

model.eval()
y_true, y_pred = [], []

with torch.no_grad():
    for images, labels in mnist_test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(labels.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

# Compute metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')
conf_matrix = confusion_matrix(y_true, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

Accuracy: 0.9862
Precision: 0.9861
Recall: 0.9860
F1 Score: 0.9861
Confusion Matrix:
[[ 972    0    3    0    0    1    3    1    0    0]
 [   0 1130    2    0    1    0    1    1    0    0]
 [   1    1 1021    0    1    0    1    5    2    0]
 [   0    0    1  995    0    7    0    2    3    2]
 [   0    0    1    0  971    0    5    0    0    5]
 [   2    0    0    9    0  877    3    1    0    0]
 [   4    2    1    0    2    2  944    0    3    0]
 [   1    2    8    2    0    0    0 1010    1    4]
 [   3    2    3    2    0    4    0    2  954    4]
 [   1    1    0    1    8    5    0    5    0  988]]
